In [8]:
# xcube_sh imports
from xcube_sh.cube import open_cube
from xcube_sh.config import CubeConfig
from xcube_sh.observers import Observers
from xcube_sh.viewer import ViewerServer
from xcube_sh.sentinelhub import SentinelHub

# xcube imports
from xcube.core.maskset import MaskSet
from xcube.core.geom import mask_dataset_by_geometry
from xcube.core.geom import clip_dataset_by_geometry
from xcube.core.evaluate import evaluate_dataset

# Various utilities
import json
from datetime import date, timedelta, datetime
import xarray as xr
import shapely.geometry
import IPython.display
import zarr
import numpy as np

import pandas as pd
import geopandas as gpd

To present the timeline of NDVI values, you will need to create a linear chart. 
 
To create the chart, it´s neccessary to import the Matplot library (matplotlib) first. The matplotlib is a plotting library which renders the plots.

In [2]:
import matplotlib.pyplot as plt

#### Specification of a geographical area of interest
The next step is to specify the area of interest - i.e. a geographical area, which the cube should cover and for which the NDVI will be calculated. 

The area of interest is specified via bounding box. You have to specify geographical coordinates of each corner of this bounding box.

For this demo, we are focussing on small coastal area near Kiel in Northern Germany (Baltic Sea).

In [3]:
x1 = 11.012421
y1 = 46.659805
x2 = 11.505432
y2 = 47.009289

In [4]:
bbox = x1, y1, x2, y2

 #### Visualisation of area of interest
 To check whether the area of interest has been specified correctly, you can integrate a simple map allowing visualisation of this area.

Visualize the bounding box.

In [5]:
IPython.display.GeoJSON(shapely.geometry.box(*bbox).__geo_interface__)

<IPython.display.GeoJSON object>

 #### Setting the spatial resolution of the cube
 The next step is to specify the spatial resolution (pixel size) on which the raster data (in this case Sentinel-2 imageries) should be resampled and stored in.
 
In this case, the desired resolution is roughly 10 meters per pixel:


In [6]:
spatial_res = 0.00018   # = 10.038 meters in degree>

#### List all the available dataset

In [9]:
SH = SentinelHub()
SH.dataset_names

['LMSSL1',
 'S3OLCI',
 'S3SLSTR',
 'S2L1C',
 'HLS',
 'LOTL2',
 'DEM',
 'LETML2',
 'LTML1',
 'LTML2',
 'S5PL2',
 'S2L2A',
 'MODIS',
 'CUSTOM',
 'LETML1',
 'S1GRD',
 'LOTL1']

#### Getting the metadata of the collection we want to use
Get the metadata of the collection we want to load. We need this information to make correct choices when subsetting our data, e.g., band names, spatial and temporal extents etc.

In [10]:
SH.band_names('S1GRD')

['VV', 'HH', 'VH', 'localIncidenceAngle', 'scatteringArea', 'shadowMask', 'HV']

#### Configuring the data content of the cube
Now we specify the properties of our cube. 
First, the satellite and processing level of data to be integrated is specified. In this case, the Sentinel-2 Level 2A data will be used.

In the second row, the spectral bands are selected, which should be integarated. As we need to calculate NDVI index, we will need red and NIR bands (B04 and B08 of Sentinel-2).

Also, the time range and time period of the cube is specified in this step. In this case, we will be interested in data acquired between 14.5. and 31.7. 2018. A custom days time period will be used.

In [10]:
start_date = date(2018, 2, 1)
end_date = date(2018, 7, 1)
bands = ['B02', 'B03', 'B04', 'B11', 'CLM'] # how to get metadata of the cube
time_period = '5D'
#time_range = [(end_date - timedelta(days=days)).strftime("%Y-%m-%d"), end_date.strftime("%Y-%m-%d")]
time_range = [start_date.strftime("%Y-%m-%d"), end_date.strftime("%Y-%m-%d")]
cube_config = CubeConfig(dataset_name='S1GRD',
                         band_names=bands,
                         tile_size=[512, 512],
                         bbox=bbox,
                         spatial_res=spatial_res,
                         time_range=time_range,
                         time_period=time_period)